In [7]:
import os
import time

os.chdir("../")
print(os.getcwd())

/Users/ravikumar/Developer/upwork/RAG-Video-Transcription


In [5]:
!mkdir -p data
!wget -P data/videos https://storage.googleapis.com/generativeai-downloads/images/GreatRedSpot.mp4

--2024-11-29 20:52:06--  https://storage.googleapis.com/generativeai-downloads/images/GreatRedSpot.mp4
Resolving storage.googleapis.com (storage.googleapis.com)... 2404:6800:4007:822::201b, 2404:6800:4007:80a::201b, 2404:6800:4007:814::201b, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|2404:6800:4007:822::201b|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 238090979 (227M) [video/mp4]
Saving to: ‘data/videos/GreatRedSpot.mp4’

GreatRedSpot.mp4    100%[===================>] 227.06M  12.0MB/s    in 20s     

2024-11-29 20:52:27 (11.2 MB/s) - ‘data/videos/GreatRedSpot.mp4’ saved [238090979/238090979]



In [2]:
import google.generativeai as genai

genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

In [3]:
video_file_name = "data/videos/GreatRedSpot.mp4"

In [4]:
print(f"Uploading file...")
video_file = genai.upload_file(path=video_file_name)
print(f"Completed upload: {video_file.uri}")

Uploading file...
Completed upload: https://generativelanguage.googleapis.com/v1beta/files/lzdkbl12fbm5


In [ ]:
EXTRACT_TRANSCRIPT_TEMPLATE = """
"""

In [26]:
# Create the prompt.
prompt = "Transcribe the audio from this video, giving timestamps for salient events in the video. "

model = genai.GenerativeModel(
    model_name="gemini-1.5-pro"
)

In [8]:
if video_file.state.name == "PROCESSING":
    time.sleep(10)
    video_file = genai.get_file(video_file.name)
if video_file.state.name == "FAILED":
    raise ValueError(video_file.state.name)

In [27]:
TRANSCRIBE_VIDEO_PROMPT = """
Transcribe the video audio into the WEBVTT format as shown below:

WEBVTT

00:00:46.000 --> 00:00:50.020
<v SpeakerName>OK, So this is our agenda for today.</v>

00:00:50.440 --> 00:00:51.050
<v SpeakerName>OK.</v>

00:00:51.710 --> 00:00:53.640
<v SpeakerName>Timewise, we're probably talking about a couple of hours.</v>

- Use exact timestamps.
- Replace "SpeakerName" with the actual speaker's name if provided.
- Maintain accuracy in both the transcription and formatting.
- Use the speaker tag format <v SpeakerName> and </v> consistently.
"""


In [28]:
response = model.generate_content(
    [video_file, TRANSCRIBE_VIDEO_PROMPT], request_options={"timeout": 1200}
)

In [29]:
response.text

"Absolutely! Here's the transcript in WEBVTT format:\n\n```webvtt\nWEBVTT\n\n00:00.000 --> 00:08.000\n<v Narrator>Jupiter is the largest and oldest planet in our solar system. Its history spans 4.5 billion years.\n\n00:10.000 --> 00:15.000\n<v Narrator>This gas giant is made of the same elements as a star, but it did not grow massive enough to ignite.\n\n00:18.000 --> 00:28.000\n<v Narrator>Jupiter's appearance is the result of its swirling interior of gases and liquids, producing a tapestry of colorful cloud bands, as well as the iconic Great Red Spot.\n\n00:33.000 --> 00:45.000\n<v Narrator>The Great Red Spot is a gigantic storm. It's an anticyclone, and with no land mass on the planet to slow it down, the Great Red Spot has raged for over a century.\n\n00:48.000 --> 00:56.000\n<v Narrator>But, scientists studying the spot have noticed that it has been changing over time. The color is deepening, and it's actually shrinking and getting rounder.\n\n00:58.000 --> 01:04.000\n<v Narrator>

In [30]:
response.text.split("\n\n")

["Absolutely! Here's the transcript in WEBVTT format:",
 '```webvtt\nWEBVTT',
 '00:00.000 --> 00:08.000\n<v Narrator>Jupiter is the largest and oldest planet in our solar system. Its history spans 4.5 billion years.',
 '00:10.000 --> 00:15.000\n<v Narrator>This gas giant is made of the same elements as a star, but it did not grow massive enough to ignite.',
 "00:18.000 --> 00:28.000\n<v Narrator>Jupiter's appearance is the result of its swirling interior of gases and liquids, producing a tapestry of colorful cloud bands, as well as the iconic Great Red Spot.",
 "00:33.000 --> 00:45.000\n<v Narrator>The Great Red Spot is a gigantic storm. It's an anticyclone, and with no land mass on the planet to slow it down, the Great Red Spot has raged for over a century.",
 "00:48.000 --> 00:56.000\n<v Narrator>But, scientists studying the spot have noticed that it has been changing over time. The color is deepening, and it's actually shrinking and getting rounder.",
 '00:58.000 --> 01:04.000\n<v N

In [ ]:
# import os
# import time
# import google.generativeai as genai
# from datetime import datetime
# import re
# from dotenv import load_dotenv

# # Load environment variables from .env file
# load_dotenv()


# def generate_vtt_filename(video_filename):
#     """Generate a VTT filename based on the video filename and current timestamp."""
#     base_name = os.path.splitext(os.path.basename(video_filename))[0]
#     timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
#     return f"{base_name}_{timestamp}.vtt"


# def format_timestamp(timestamp):
#     """Format timestamp to ensure exactly 3 decimal places."""
#     # Split into seconds and milliseconds
#     parts = timestamp.split('.')
#     seconds = parts[0]
#     # Ensure exactly 3 decimal places
#     msec = parts[1][:3] if len(parts) > 1 else "000"
#     msec = msec.ljust(3, '0')
#     return f"{seconds}.{msec}"


# def clean_vtt_content(content):
#     """Clean and format the VTT content."""
#     # Remove any markdown code block markers and extra text
#     content = re.sub(r'```.*?\n', '', content)
#     content = re.sub(r'```', '', content)
#     content = re.sub(r'WEBVTT file:', 'WEBVTT', content)
    
#     # Start with WEBVTT header
#     formatted_lines = ['WEBVTT', '']
    
#     # Split content into lines and process
#     lines = content.strip().split('\n')
    
#     # Skip any existing WEBVTT header
#     i = 0
#     while i < len(lines) and (not lines[i].strip() or lines[i].strip().startswith('WEBVTT')):
#         i += 1
    
#     # Process remaining lines
#     while i < len(lines):
#         line = lines[i].strip()
        
#         # Skip empty lines
#         if not line:
#             i += 1
#             continue
            
#         # Process timestamp lines
#         if '-->' in line:
#             # Format timestamps
#             times = line.split(' --> ')
#             if len(times) == 2:
#                 # Ensure HH:MM:SS format for timestamps
#                 start_time = format_timestamp(times[0].strip())
#                 end_time = format_timestamp(times[1].strip())
#                 if ':' not in start_time:
#                     start_time = f"00:{start_time}"
#                 if ':' not in end_time:
#                     end_time = f"00:{end_time}"
#                 formatted_lines.append(f"{start_time} --> {end_time}")
                
#                 # Process speaker line
#                 if i + 1 < len(lines):
#                     speaker_line = lines[i + 1].strip()
#                     if speaker_line:
#                         formatted_lines.append(speaker_line)
#                         formatted_lines.append('')  # Single blank line between entries
#                     i += 2
#                     continue
#         i += 1
    
#     # Remove trailing empty lines and ensure single newline at end
#     while formatted_lines and not formatted_lines[-1]:
#         formatted_lines.pop()
    
#     return '\n'.join(formatted_lines) + '\n'


# def save_vtt_content(content, output_path):
#     """Save the VTT content to a file."""
#     with open(output_path, "w", encoding="utf-8") as f:
#         f.write(content)
#     print(f"Saved transcript to: {output_path}")


# def transcribe_video(video_file_path, output_dir="data/transcripts"):
#     """Transcribe video using Gemini and save as VTT file."""
#     # Configure Gemini
#     genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

#     # Upload video file
#     print(f"Uploading file: {video_file_path}")
#     video_file = genai.upload_file(path=video_file_path)
#     print(f"Completed upload: {video_file.uri}")

#     # Check whether the file is ready to be used.
#     while video_file.state.name == "PROCESSING":
#         print(".", end="")
#         time.sleep(10)
#         video_file = genai.get_file(video_file.name)

#     if video_file.state.name == "FAILED":
#         raise ValueError(video_file.state.name)

#     # Initialize model
#     model = genai.GenerativeModel(model_name="gemini-1.5-pro")

#     # Define transcription prompt
#     TRANSCRIBE_VIDEO_PROMPT = """
#     Transcribe the video audio into WebVTT format following these exact rules:
#     1. Start with only "WEBVTT" on the first line (no extra text)
#     2. Add exactly one blank line after WEBVTT
#     3. For each segment:
#        - Use timestamps in format HH:MM:SS.mmm (exactly 3 decimal places)
#        - Format: XX:XX:XX.XXX --> XX:XX:XX.XXX
#        - Use speaker tags: <v Speaker>text</v>
#        - Add exactly one blank line between segments
#     4. Keep segments short (around 5-10 seconds each)
#     5. Do not add any extra text or formatting

#     Example format:
#     WEBVTT

#     00:00:46.000 --> 00:00:50.020
#     <v Speaker>Text content goes here.</v>

#     00:00:50.440 --> 00:00:51.050
#     <v Speaker>Next segment text.</v>
#     """

#     try:
#         # Generate transcription
#         print("Generating transcription...")
#         response = model.generate_content(
#             [video_file, TRANSCRIBE_VIDEO_PROMPT], request_options={"timeout": 1200}
#         )

#         # Clean and format the response
#         formatted_content = clean_vtt_content(response.text)

#         # Create output directory if it doesn't exist
#         os.makedirs(output_dir, exist_ok=True)

#         # Generate output filename and save
#         output_file = os.path.join(output_dir, generate_vtt_filename(video_file_path))
#         save_vtt_content(formatted_content, output_file)

#         return output_file
#     except Exception as e:
#         print(f"Error during transcription: {str(e)}")
#         raise


# if __name__ == "__main__":
#     # Example usage
#     video_file_path = "data/videos/GreatRedSpot.mp4"
#     transcribe_video(video_file_path)


In [ ]:
# from dotenv import load_dotenv
# import os
# import time
# from elasticsearch import Elasticsearch
# from typing import List, Dict, Optional
# import json
# from datetime import datetime
# from transcript_processor import TranscriptProcessor
# from pathlib import Path

# load_dotenv()

# from langchain_openai import OpenAIEmbeddings
# from langchain_elasticsearch import ElasticsearchStore
# from langchain.schema import Document

# class VideoTranscriptionStore:
#     def __init__(self, videos_dir: str, transcripts_dir: str):
#         self.vector_store = self.init_vector_store()
#         self.processor = TranscriptProcessor(videos_dir, transcripts_dir)
    
#     def init_vector_store(self) -> ElasticsearchStore:
#         # Initialize OpenAI embeddings
#         embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
        
#         # Wait for Elasticsearch to be ready
#         es_client = Elasticsearch(
#             "http://localhost:9200",
#             basic_auth=("elastic", "changeme")
#         )
        
#         # Wait for up to 30 seconds for Elasticsearch to be ready
#         start_time = time.time()
#         while time.time() - start_time < 30:
#             try:
#                 if es_client.ping():
#                     print("Successfully connected to Elasticsearch")
#                     break
#             except Exception as e:
#                 print(f"Waiting for Elasticsearch to be ready... {str(e)}")
#                 time.sleep(2)
#         else:
#             raise Exception("Could not connect to Elasticsearch after 30 seconds")

#         # Initialize vector store
#         return ElasticsearchStore(
#             es_url="http://localhost:9200",
#             index_name="video_transcriptions",
#             embedding=embeddings,
#             es_user="elastic",
#             es_password="changeme",
#         )

#     def upsert_video(self, video_filename: str) -> None:
#         """
#         Process and upsert a single video's transcription.
        
#         Args:
#             video_filename: Name of the video file
#         """
#         try:
#             # Process the video and its transcript
#             result = self.processor.process_video(video_filename)
            
#             documents = []
#             for segment in result["segments"]:
#                 # Create a Document object with the segment text and all metadata
#                 doc = Document(
#                     page_content=segment["text"],
#                     metadata={
#                         **result["metadata"],  # Include all video/transcript metadata
#                         "start_time": segment["start_time"],
#                         "end_time": segment["end_time"],
#                     }
#                 )
#                 documents.append(doc)
            
#             # Add documents to the vector store
#             self.vector_store.add_documents(documents)
#             print(f"Added {len(documents)} segments from video: {video_filename}")
            
#         except Exception as e:
#             print(f"Error processing video {video_filename}: {str(e)}")

#     def upsert_all_videos(self) -> None:
#         """Process and upsert all videos in the videos directory."""
#         videos_dir = Path(self.processor.videos_dir)
#         for video_file in videos_dir.glob("*.mp4"):
#             self.upsert_video(video_file.name)

#     def search_transcriptions(
#         self,
#         query: str,
#         filter_metadata: Optional[Dict] = None,
#         k: int = 5
#     ) -> List[Dict]:
#         """
#         Search transcriptions using a natural language query.
        
#         Args:
#             query: Search query
#             filter_metadata: Optional metadata filters (e.g., {"video_filename": "example.mp4"})
#             k: Number of results to return
            
#         Returns:
#             List of relevant transcription segments with metadata
#         """
#         # Perform the search
#         results = self.vector_store.similarity_search_with_score(
#             query,
#             k=k,
#             filter=filter_metadata
#         )
        
#         # Format the results
#         formatted_results = []
#         for doc, score in results:
#             result = {
#                 "text": doc.page_content,
#                 "start_time": doc.metadata["start_time"],
#                 "end_time": doc.metadata["end_time"],
#                 "video_filename": doc.metadata["video_filename"],
#                 "relevance_score": score,
#             }
            
#             # Add all other metadata
#             result["metadata"] = {k: v for k, v in doc.metadata.items() 
#                                 if k not in ["start_time", "end_time", "video_filename"]}
            
#             formatted_results.append(result)
        
#         return formatted_results

# def main():
#     # Initialize the store with your video and transcript directories
#     store = VideoTranscriptionStore(
#         videos_dir="/Users/ravikumar/Developer/upwork/RAG-Video-Transcription/data/videos",
#         transcripts_dir="/Users/ravikumar/Developer/upwork/RAG-Video-Transcription/data/transcripts"
#     )
    
#     # Upsert all videos
#     store.upsert_all_videos()
    
#     # Example search
#     results = store.search_transcriptions(
#         query="What is discussed about python tutorial?",
#         k=3
#     )
    
#     # Print results
#     print("\nSearch Results:")
#     for result in results:
#         print(f"\nSegment: {result['text']}")
#         print(f"Video: {result['video_filename']}")
#         print(f"Timestamp: {result['start_time']} - {result['end_time']}")
#         print(f"Relevance Score: {result['relevance_score']}")
#         print("Additional Metadata:", json.dumps(result["metadata"], indent=2))

# if __name__ == "__main__":
#     main()

In [32]:
# import os
# import re
# from typing import List, Dict, Optional
# from datetime import datetime
# import webvtt
# from pathlib import Path

# class TranscriptSegment:
#     def __init__(self, text: str, start: str, end: str):
#         self.text = text
#         self.start = start
#         self.end = end
        
#     def to_dict(self) -> Dict:
#         return {
#             "text": self.text,
#             "start_time": self.start,
#             "end_time": self.end
#         }

# class TranscriptProcessor:
#     def __init__(self, videos_dir: str, transcripts_dir: str):
#         self.videos_dir = Path(videos_dir)
#         self.transcripts_dir = Path(transcripts_dir)
    
#     def parse_vtt(self, vtt_path: str) -> List[TranscriptSegment]:
#         """Parse a VTT file into segments."""
#         segments = []
#         for caption in webvtt.read(vtt_path):
#             # Clean the text: remove multiple spaces and newlines
#             text = ' '.join(caption.text.split())
#             segments.append(TranscriptSegment(
#                 text=text,
#                 start=caption.start,
#                 end=caption.end
#             ))
#         return segments
    
#     def extract_metadata(self, video_path: str, transcript_path: str) -> Dict:
#         """Extract metadata from video and transcript files."""
#         video_path = Path(video_path)
#         transcript_path = Path(transcript_path)
        
#         # Get video information
#         video_stats = video_path.stat()
        
#         # Extract timestamp from transcript filename if it exists
#         # Format: filename_YYYYMMDD_HHMMSS.vtt
#         timestamp_match = re.search(r'_(\d{8}_\d{6})\.vtt$', transcript_path.name)
#         processed_date = None
#         if timestamp_match:
#             date_str = timestamp_match.group(1)
#             try:
#                 processed_date = datetime.strptime(date_str, '%Y%m%d_%H%M%S').isoformat()
#             except ValueError:
#                 pass
        
#         metadata = {
#             "video_filename": video_path.name,
#             "video_size_bytes": video_stats.st_size,
#             "video_created_at": datetime.fromtimestamp(video_stats.st_ctime).isoformat(),
#             "video_modified_at": datetime.fromtimestamp(video_stats.st_mtime).isoformat(),
#             "transcript_filename": transcript_path.name,
#             "transcript_processed_at": processed_date,
#             "file_extension": video_path.suffix.lower(),
#         }
        
#         return metadata
    
#     def find_matching_transcript(self, video_filename: str) -> Optional[Path]:
#         """Find the matching transcript file for a video."""
#         base_name = Path(video_filename).stem
#         matching_transcripts = list(self.transcripts_dir.glob(f"{base_name}*.vtt"))
        
#         if not matching_transcripts:
#             return None
        
#         # Return the most recent transcript if multiple exist
#         return sorted(matching_transcripts, key=lambda x: x.stat().st_mtime)[-1]
    
#     def process_video(self, video_filename: str) -> Optional[Dict]:
#         """Process a single video and its transcript."""
#         video_path = self.videos_dir / video_filename
#         if not video_path.exists():
#             raise FileNotFoundError(f"Video file not found: {video_filename}")
        
#         transcript_path = self.find_matching_transcript(video_filename)
#         if not transcript_path:
#             raise FileNotFoundError(f"No matching transcript found for video: {video_filename}")
        
#         # Parse the transcript
#         segments = self.parse_vtt(str(transcript_path))
        
#         # Extract metadata
#         metadata = self.extract_metadata(video_path, transcript_path)
        
#         return {
#             "segments": [seg.to_dict() for seg in segments],
#             "metadata": metadata
#         }
    
#     def process_all_videos(self) -> List[Dict]:
#         """Process all videos in the videos directory."""
#         results = []
#         for video_file in self.videos_dir.glob("*.mp4"):
#             try:
#                 result = self.process_video(video_file.name)
#                 if result:
#                     results.append(result)
#             except Exception as e:
#                 print(f"Error processing {video_file.name}: {str(e)}")
#         return results


00:00:00.000
00:00:04.980
Jupiter is the largest and oldest planet in our solar system.
#########################
00:00:04.980
00:00:08.940
Its history spans 4.5 billion years.
#########################
00:00:08.940
00:00:15.900
This gas giant is made of the same elements as a star, but it did not grow massive enough to ignite.
#########################
00:00:15.900
00:00:21.840
Jupiter's appearance is the result of its swirling interior of gases and liquids,
#########################
00:00:21.840
00:00:28.740
producing a tapestry of colorful cloud bands, as well as the iconic Great Red Spot.
#########################
00:00:28.740
00:00:31.770

#########################
00:00:31.770
00:00:41.730
The Great Red Spot is a gigantic storm. It's an anticyclone, and with no land mass on the planet to slow it down, the Great Red Spot has raged for over a century.
#########################
00:00:41.730
00:00:45.720

#########################
00:00:45.720
00:00:47.700

#########################


In [ ]:
"""
WEBVTT



97afec20-7bee-4318-a470-6822ee7ddbe7/10-0



00:00:03.776 --> 00:00:04.816



<v Lastname, Firstname Middlename1>I have to go back and look at



it.</v>



97afec20-7bee-4318-a470-6822ee7ddbe7/14-0



00:00:04.816 --> 00:00:08.296



<v Lastname, Firstname Middlename1>Typically you also get give me a



second.</v>



97afec20-7bee-4318-a470-6822ee7ddbe7/20-0



00:00:08.856 --> 00:00:11.016



<v Lastname, Firstname Middlename1>Is there a multi document which



I can share?</v>



97afec20-7bee-4318-a470-6822ee7ddbe7/27-0



00:00:12.576 --> 00:00:14.176



<v Lastname, Firstname Middlename1>There was an MRD document



somewhere.</v>



97afec20-7bee-4318-a470-6822ee7ddbe7/35-0



00:00:20.526 --> 00:00:23.766



<v Lastname, Firstname Middlename2>Hsds is a is like the ground



truth.</v>



97afec20-7bee-4318-a470-6822ee7ddbe7/41-0



00:00:25.766 --> 00:00:26.926



<v Lastname, Firstname Middlename1>Yeah, that's the ground truth.</v>



97afec20-7bee-4318-a470-6822ee7ddbe7/47-0



00:00:26.716 --> 00:00:29.756



<v Lastname, Firstname Middlename2>From where you create ppts and



documents and everything.</v>



97afec20-7bee-4318-a470-6822ee7ddbe7/59-0



00:00:30.036 --> 00:00:33.600



<v Lastname, Firstname Middlename1>Yeah, yeah, yeah. That is the



one which has to very clearly</v>



97afec20-7bee-4318-a470-6822ee7ddbe7/52-0



00:00:31.906 --> 00:00:32.066



<v Lastname, Firstname Middlename2>OK.</v>
"""